In [41]:
pip install XlsxWriter


  Obtaining dependency information for XlsxWriter from https://files.pythonhosted.org/packages/a7/ea/53d1fe468e63e092cf16e2c18d16f50c29851242f9dd12d6a66e0d7f0d02/XlsxWriter-3.2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/159.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/159.9 kB ? eta -:--:--
   ------- ------------------------------- 30.7/159.9 kB 262.6 kB/s eta 0:00:01
   --------- ----------------------------- 41.0/159.9 kB 245.8 kB/s eta 0:00:01
   ----------------------------- -------- 122.9/159.9 kB 554.9 kB/s eta 0:00:01
   -------------------------------------- 159.9/159.9 kB 639.7 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [82]:
#import all required libraries.

from bs4 import BeautifulSoup
import pandas as pd
import requests
import xlsxwriter

In [83]:
def getContent(url):
    try:
        HEADERS = ({'User-Agent':
           'add your own :)',
                           'Accept-Language': 'en-US, en;q=0.5'})
        page = requests.get(url, headers=HEADERS)
    
        if page.status_code != 200:
            print("Check the URL")
        else:
            soup = BeautifulSoup(page.content, 'html.parser')
    except ConnectionError:
        soup = ''
        
    
    return soup;


def getInfo(tag,attributeType, attributName,new_soup):
    
    try:
    
        info = new_soup.find(tag, attrs={attributeType:attributName})
    
        info = info.text
    
        info = info.strip()
    except AttributeError:
        info = ''
    
    return info

In [87]:
url = 'https://www.amazon.com/s?k=lotion&crid=W9CWRPHCAWCS&sprefix=lotion%2Caps%2C564&ref=nb_sb_noss_1'
soup = getContent(url)

    

In [88]:
#getting all the links.

links = soup.find_all('a', attrs={'class':'a-link-normal s-no-outline'})


In [89]:
#now we take one by one all the links and extract title, price and rating of product.
link_list = []
for link in links:
    link_list.append(link.get("href"))

    
titles = []
prices = []
ratings = []
i = 0;
for link in link_list:
   
    HEADERS = ({'User-Agent':'add your own :)',
                           'Accept-Language': 'en-US, en;q=0.5'})
    page1 = requests.get("https://www.amazon.com" + link, headers=HEADERS)
    
   
    new_soup = BeautifulSoup(page1.content, 'html.parser')
    
    #get title of the product.
    tag = 'span'
    attributeType = 'id'
    attributeName = 'productTitle'
    title = getInfo(tag,attributeType, attributeName,new_soup)
   
    titles.append(title)
    


    #get price of the product.
    tag = 'span'
    attributeType = 'class'
    attributeName = 'aok-offscreen'
    price = getInfo(tag,attributeType, attributeName,new_soup)
    prices.append(price)

    #get rating of the product.
    tag = 'span'
    attributeType = 'class'
    attributeName = 'a-icon-alt'
    rating = getInfo(tag,attributeType, attributeName,new_soup)
    ratings.append(rating)
    if i == 10:
        break
    i += 1
    
   


In [90]:

workbook = xlsxwriter.Workbook('cream1.xlsx')
worksheet = workbook.add_worksheet()


worksheet.write(0,0,"Title")
worksheet.write(0,1,"Price")
worksheet.write(0,2,"Rating")

row = 1
for t, p , r in zip(titles, prices, ratings):
    worksheet.write(row, 0, t) 
    worksheet.write(row, 1, p) 
    worksheet.write(row, 2, r) 
    row += 1
workbook.close()






In [91]:
data = pd.read_excel('cream1.xlsx') 
data.head()

,Title,Price,Rating
0,"Amazon Basics Cocoa Butter Body Lotion, Lightl...",$0.20 per Fl Oz,4.4 out of 5 stars
1,"Aveeno Daily Moisturizer, Body Lotion, For Dry...",$0.59 per Fl Oz,4.7 out of 5 stars
2,Aveeno Daily Stress Relief Body Lotion with La...,$0.75 per Fl Oz,4.7 out of 5 stars
3,Aveeno Skin Relief Fragrance-Free Moisturizing...,$0.59 per Fl Oz,4.7 out of 5 stars
4,"eos Shea Better Body Lotion- Vanilla Cashmere,...",$0.56 per Fl Oz,4.7 out of 5 stars
